In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
!git clone https://github.com/open-mmlab/mmsegmentation.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
     |████████████████████████████████| 46.6 MB 16.0 MB/s 
     |████████████████████████████████| 190 kB 8.1 MB/s 
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 7190, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 7190 (delta 51), reused 162 (delta 38), pack-reused 6995
Receiving objects: 100% (7190/7190), 13.43 MiB | 7.83 MiB/s, done.
Resolving deltas: 100% (5244/5244), done.


In [3]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0+cu113 (Tesla P100-PCIE-16GB)


In [4]:
%cd mmsegmentation

/content/mmsegmentation


In [5]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
import mmcv

In [6]:
import os.path as osp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
!mkdir checkpoints
!wget -O //content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth

--2022-05-30 03:07:57--  https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.72
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328290585 (313M) [application/octet-stream]
Saving to: ‘//content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth’

//content/mmsegment 100%[===================>] 313.08M  9.84MB/s    in 35s     

2022-05-30 03:08:33 (8.98 MB/s) - ‘//content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth’ saved [328290585/328290585]



In [8]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

classes = ('background', 'road')
palette = [[0, 0, 0], [255, 255, 0]]

@DATASETS.register_module()
class SIADataset(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.png', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None

In [9]:
# config 파일을 설정하고, 다운로드 받은 pretrained 모델을 checkpoint로 설정. 
config_file = '/content/mmsegmentation/configs/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes.py'
checkpoint_file = '/content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth'

from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='MixVisionTransformer',
        in_channels=3,
        embed_dims=64,
        num_stages=4,
        num_layers=[3, 6, 40, 3],
        num_heads=[1, 2, 5, 8],
        patch_sizes=[7, 3, 3, 3],
        sr_ratios=[8, 4, 2, 1],
        out_indices=(0, 1, 2, 3),
        mlp_ratio=4,
        qkv_bias=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.1,
        init_cfg=dict(type='Pretrained', checkpoint='pretrain/mit_b5.pth')),
    decode_head=dict(
        type='SegformerHead',
        in_channels=[64, 128, 320, 512],
        in_index=[0, 1, 2, 3],
        channels=256,
        dropout_ratio=0.1,
        num_classes=19,
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    train_cf

In [10]:
cfg.norm_cfg = dict(type='BN', requires_grad=True)
#cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg

cfg.model.decode_head.num_classes = 2

cfg.device='cuda'

cfg.model.decode_head.loss_decode = dict(type='DiceLoss', loss_weight = 1.0)

In [11]:
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (512, 512)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(1024, 1024), ratio_range=(0.5, 1.5)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.val_pipeline = [
                    dict(type='LoadImageFromFile'),
                    dict(
                        type='MultiScaleFlipAug',
                        img_scale=(1024, 1024),
                        flip=False,
                        transforms=[
                                    dict(type='Resize', keep_ratio=True),
                                    dict(type='RandomFlip'),
                                    dict(
                                        type='Normalize',
                                        mean=[123.675, 116.28, 103.53],
                                        std=[58.395, 57.12, 57.375],
                                        to_rgb=True),
                                    dict(type='ImageToTensor', keys=['img']),
                                    dict(type='Collect', keys=['img'])
                                    ]),
                    
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024),
        img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            #dict(type='ResizeToMultiple', size_divisor=32),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

In [12]:
cfg.dataset_type = 'SIADataset'
cfg.data_root = '/content/drive/MyDrive/SIA'

cfg.data.train.type = 'SIADataset'
cfg.data.train.data_root = '/content/drive/MyDrive/SIA'
cfg.data.train.img_dir = 'Data_set/road_dataset/img_dir/train'
cfg.data.train.ann_dir = 'Data_set/road_dataset/ann_01_dir/train'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = '/content/drive/MyDrive/SIA/Data_set/road_dataset/mask_dir/train/up_train_road.txt'

cfg.data.val.type = 'SIADataset'
cfg.data.val.data_root = '/content/drive/MyDrive/SIA'
cfg.data.val.img_dir = 'Data_set/road_dataset/img_dir/train'
cfg.data.val.ann_dir = 'Data_set/road_dataset/ann_01_dir/train'
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = '/content/drive/MyDrive/SIA/Data_set/road_dataset/mask_dir/train/up_val_road.txt'

cfg.data.test.type = 'SIADataset'
cfg.data.test.data_root = '/content/drive/MyDrive/SIA'
cfg.data.test.img_dir = 'Data_set/road_dataset/img_dir/val'
cfg.data.test.ann_dir = 'Data_set/road_dataset/ann_01_dir/val'
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = '/content/drive/MyDrive/SIA/Data_set/road_dataset/mask_dir/val/up.txt'

In [13]:
'''
cfg.data.train.type = 'SIADataset'
cfg.data.train.data_root = '/content/drive/MyDrive/SIA'
cfg.data.train.img_dir = 'images/up025'
cfg.data.train.ann_dir = 'labels'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = '/content/drive/MyDrive/SIA/splits/road/train_road.txt'
'''

"\ncfg.data.train.type = 'SIADataset'\ncfg.data.train.data_root = '/content/drive/MyDrive/SIA'\ncfg.data.train.img_dir = 'images/up025'\ncfg.data.train.ann_dir = 'labels'\ncfg.data.train.pipeline = cfg.train_pipeline\ncfg.data.train.split = '/content/drive/MyDrive/SIA/splits/road/train_road.txt'\n"

In [14]:
cfg.load_from = '/content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth'

# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive/SIA/semantic_checkpoint/Segformer/road/DFL_class2'

In [15]:
cfg.runner.max_iteTrs = 200
cfg.log_config.interval = 100
cfg.evaluation.interval = 1000  # 모델 학습시 평가를 몇 번째 iteration마다 할 것인지 지정
cfg.checkpoint_config.interval = 1000  # 모델 학습시 학습한 모델을 몇 번째 iteration마다 저장할 것인지 지정

cfg.runner = dict(type='IterBasedRunner', max_iters=20000)  # Iteration으로 동작, Epoch로 동작하게 변경할 수도 있음
# cfg.runner = dict(type='EpochBasedRunner', max_epochs=4000)  # Epoch로 변경
cfg.workflow = [('train', 1)]

# Set seed to facitate reproducing the result
cfg.seed = 0
#set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='MixVisionTransformer',
        in_channels=3,
        embed_dims=64,
        num_stages=4,
        num_layers=[3, 6, 40, 3],
        num_heads=[1, 2, 5, 8],
        patch_sizes=[7, 3, 3, 3],
        sr_ratios=[8, 4, 2, 1],
        out_indices=(0, 1, 2, 3),
        mlp_ratio=4,
        qkv_bias=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.1,
        init_cfg=dict(type='Pretrained', checkpoint='pretrain/mit_b5.pth')),
    decode_head=dict(
        type='SegformerHead',
        in_channels=[64, 128, 320, 512],
        in_index=[0, 1, 2, 3],
        channels=256,
        dropout_ratio=0.1,
        num_classes=2,
        norm_cfg=dict(type='BN', requires_grad=True),
        align_corners=False,
        loss_decode=dict(type='DiceLoss', loss_weight=1.0)),
    train_cfg=dict(),
    test_cfg=dict(mode='slide',

### 모델 학습

In [16]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True,
                meta=dict(CLASSES=classes, PALETTE=palette))

2022-05-30 03:11:28,780 - mmseg - INFO - Loaded 542 images
2022-05-30 03:11:41,473 - mmseg - INFO - Loaded 66 images
2022-05-30 03:11:41,477 - mmseg - INFO - load checkpoint from local path: /content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth
2022-05-30 03:11:41,989 - mmseg - WARNING - The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([19, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 256, 1, 1]).
size mismatch for decode_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
2022-05-30 03:11:41,997 - mmseg - INFO - Start running, host: root@c56660545965, work_dir: /content/drive/MyDrive/SIA/semantic_checkpoint/Segformer/road/DFL_class2
2022-05-30 03:11:41,999 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 0.2 task/s, elapsed: 273s, ETA:     0s

2022-05-30 03:25:43,482 - mmseg - INFO - per class results:
2022-05-30 03:25:43,484 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 90.07 | 95.11 |
|    road    | 58.82 |  72.8 |
+------------+-------+-------+
2022-05-30 03:25:43,486 - mmseg - INFO - Summary:
2022-05-30 03:25:43,488 - mmseg - INFO - 
+------+-------+-------+
| aAcc |  mIoU |  mAcc |
+------+-------+-------+
| 91.3 | 74.44 | 83.95 |
+------+-------+-------+
2022-05-30 03:25:43,491 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9130, mIoU: 0.7444, mAcc: 0.8395, IoU.background: 0.9007, IoU.road: 0.5882, Acc.background: 0.9511, Acc.road: 0.7280
2022-05-30 03:26:26,075 - mmseg - INFO - Iter [1100/20000]	lr: 4.154e-05, eta: 4:13:00, time: 3.171, data_time: 2.775, memory: 3323, decode.loss_dice: 0.1719, decode.acc_seg: 88.4176, loss: 0.1719
2022-05-30 03:27:06,427 - mmseg - INFO - Iter [1200/20000]	lr: 4.508e-05, eta: 4:01:13, time: 0.404, data_time: 0.009

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 0.3 task/s, elapsed: 253s, ETA:     0s

2022-05-30 03:37:01,161 - mmseg - INFO - per class results:
2022-05-30 03:37:01,162 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 91.03 |  97.5 |
|    road    | 58.36 | 65.46 |
+------------+-------+-------+
2022-05-30 03:37:01,164 - mmseg - INFO - Summary:
2022-05-30 03:37:01,166 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 92.03 | 74.69 | 81.48 |
+-------+-------+-------+
2022-05-30 03:37:01,169 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9203, mIoU: 0.7469, mAcc: 0.8148, IoU.background: 0.9103, IoU.road: 0.5836, Acc.background: 0.9750, Acc.road: 0.6546
2022-05-30 03:37:41,120 - mmseg - INFO - Iter [2100/20000]	lr: 5.370e-05, eta: 3:41:24, time: 2.937, data_time: 2.546, memory: 3323, decode.loss_dice: 0.1402, decode.acc_seg: 90.6921, loss: 0.1402
2022-05-30 03:38:23,236 - mmseg - INFO - Iter [2200/20000]	lr: 5.340e-05, eta: 3:35:50, time: 0.421, data_time: 

...

2022-05-30 06:36:02,481 - mmseg - INFO - per class results:
2022-05-30 06:36:02,484 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 92.06 | 96.71 |
|    road    | 65.07 | 75.47 |
+------------+-------+-------+
2022-05-30 06:36:02,485 - mmseg - INFO - Summary:
2022-05-30 06:36:02,488 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 93.09 | 78.57 | 86.09 |
+-------+-------+-------+
2022-05-30 06:36:02,491 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9309, mIoU: 0.7857, mAcc: 0.8609, IoU.background: 0.9206, IoU.road: 0.6507, Acc.background: 0.9671, Acc.road: 0.7547
2022-05-30 06:36:42,729 - mmseg - INFO - Iter [18100/20000]	lr: 5.703e-06, eta: 0:21:31, time: 2.930, data_time: 2.535, memory: 3323, decode.loss_dice: 0.0852, decode.acc_seg: 94.0236, loss: 0.0852
2022-05-30 06:37:22,843 - mmseg - INFO - Iter [18200/20000]	lr: 5.403e-06, eta: 0:20:20, time: 0.401, data_time

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 0.3 task/s, elapsed: 252s, ETA:     0s

2022-05-30 06:47:22,905 - mmseg - INFO - per class results:
2022-05-30 06:47:22,907 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 92.16 | 96.75 |
|    road    | 65.45 | 75.79 |
+------------+-------+-------+
2022-05-30 06:47:22,915 - mmseg - INFO - Summary:
2022-05-30 06:47:22,917 - mmseg - INFO - 
+-------+------+-------+
|  aAcc | mIoU |  mAcc |
+-------+------+-------+
| 93.17 | 78.8 | 86.27 |
+-------+------+-------+
2022-05-30 06:47:22,921 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9317, mIoU: 0.7880, mAcc: 0.8627, IoU.background: 0.9216, IoU.road: 0.6545, Acc.background: 0.9675, Acc.road: 0.7579
2022-05-30 06:48:03,673 - mmseg - INFO - Iter [19100/20000]	lr: 2.703e-06, eta: 0:10:11, time: 2.933, data_time: 2.536, memory: 3323, decode.loss_dice: 0.0762, decode.acc_seg: 94.2240, loss: 0.0762
2022-05-30 06:48:44,109 - mmseg - INFO - Iter [19200/20000]	lr: 2.403e-06, eta: 0:09:02, time: 0.404, data_time: 0.0

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 0.3 task/s, elapsed: 252s, ETA:     0s

2022-05-30 06:58:40,824 - mmseg - INFO - per class results:
2022-05-30 06:58:40,825 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 92.14 | 96.46 |
|    road    | 65.87 | 77.21 |
+------------+-------+-------+
2022-05-30 06:58:40,827 - mmseg - INFO - Summary:
2022-05-30 06:58:40,829 - mmseg - INFO - 
+-------+------+-------+
|  aAcc | mIoU |  mAcc |
+-------+------+-------+
| 93.17 | 79.0 | 86.83 |
+-------+------+-------+
2022-05-30 06:58:40,831 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9317, mIoU: 0.7900, mAcc: 0.8683, IoU.background: 0.9214, IoU.road: 0.6587, Acc.background: 0.9646, Acc.road: 0.7721


### 모델 결과 시각화

In [17]:
img_result = ['BLD00002_PS3_K3A_NIA0276.png',
              'BLD00010_PS3_K3A_NIA0276.png',
              'BLD00047_PS3_K3A_NIA0276.png',
              'BLD12071_PS3_K3A_NIA0391.png',
              'BLD11907_PS3_K3A_NIA0391.png',
              'BLD11611_PS3_K3A_NIA0390.png',
              'BLD11474_PS3_K3A_NIA0390.png',
              'BLD10413_PS3_K3A_NIA0388.png',
              'BLD10150_PS3_K3A_NIA0388.png',
              'BLD00836_PS3_K3A_NIA0277.png']

checkpoint_file = '/content/drive/MyDrive/SIA/semantic_checkpoint/Segformer/road/DFL_class2/iter_20000.pth'  #학습된 모델


# checkpoint 저장된 model 파일을 이용하여 모델을 생성, 이때 Config는 위에서 update된 config 사용. 
model_ckpt = init_segmentor(cfg, checkpoint_file, device='cuda:0')  #cuda gpu 사용

for ir in range(len(img_result)):
  img = mmcv.imread('/content/drive/MyDrive/SIA/Data_set/road_dataset/img_dir/val/'+img_result[ir])
  result = inference_segmentor(model_ckpt, img)
  show_result_pyplot(model_ckpt, img, result, palette)


Output hidden; open in https://colab.research.google.com to view.

In [18]:
import os
file_list = os.listdir('/content/drive/MyDrive/SIA/Data_set/road_dataset/img_dir/val')
test_png = []
for i in file_list:
  if '.png' in i:
    test_png.append(i)

for j in test_png:
  img = mmcv.imread('/content/drive/MyDrive/SIA/Data_set/road_dataset/img_dir/val/'+j)
  result = inference_segmentor(model_ckpt, img)
  show_result_pyplot(model_ckpt, img, result, palette)

Output hidden; open in https://colab.research.google.com to view.